# Santander Value Prediction Challenge
### I participated in this Kaggle competition as a tryout. Using Kera in this case
### Please go to https://www.kaggle.com/c/santander-value-prediction-challenge to download the dataset

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
training_data = pd.read_csv('input/train.csv')
test_data = pd.read_csv('input/test.csv')

In [3]:
import tensorflow as tf

In [4]:
tf.reset_default_graph()

In [24]:
x_data = training_data[[
    'f190486d6', 'eeb9cd3aa', '58e2e02e6', 'b43a7cfd5', '58232a6fb',
    '15ace8c9f', '9fd594eec', '20aa07010', '024c577b9', '00f844fea',
    '1702b5bf0', '58e056e12', '6eef030c1', 'f74e8f13d', '26fc93eb7', 
    'd6bb78916', '1c71183bb', '963a49cdc', '4edc3388d', 'fb0f5dbfe'
]]
# x_data = training_data.drop(["ID", "target"], axis=1)

In [26]:
x_test = test_data[[
    'f190486d6', 'eeb9cd3aa', '58e2e02e6', 'b43a7cfd5', '58232a6fb',
    '15ace8c9f', '9fd594eec', '20aa07010', '024c577b9', '00f844fea',
    '1702b5bf0', '58e056e12', '6eef030c1', 'f74e8f13d', '26fc93eb7', 
    'd6bb78916', '1c71183bb', '963a49cdc', '4edc3388d', 'fb0f5dbfe'
]]
# x_test = test_data.drop(["ID"], axis=1)

In [27]:
x_data_log = np.log1p(x_data)
x_test_log = np.log1p(x_test)

In [28]:
X_all = pd.concat((x_test_log, x_data_log), axis=0).replace(0,  np.nan) # exclude zeros from mean/std calculation

In [29]:
# Scale features
X_train_scala = (x_data_log - X_all.mean()) / X_all.std()
X_test_scala = (x_test_log - X_all.mean()) / X_all.std()

In [30]:
labels = np.log1p(training_data['target'])

In [31]:
from sklearn.model_selection import train_test_split

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X_train_scala,labels,test_size=0.3, random_state=101)

In [33]:
from tensorflow.contrib.keras import models

In [34]:
dnn_keras_model = models.Sequential()

In [35]:
from tensorflow.contrib.keras import optimizers
from tensorflow.contrib.keras import layers

In [36]:
input_layer = layers.Input(shape=(X_train.shape[1],))
x = input_layer
x = layers.Dense(1024, activation='linear')(x)
x = layers.LeakyReLU()(x)
#x = Dropout(0.5)(x)
x = layers.Dense(512, activation='linear')(x)
x = layers.LeakyReLU()(x)
#x = Dropout(0.1)(x)
x = layers.Dense(64, activation='relu')(x)
x = layers.Dense(1, activation='linear')(x)
keras_nn = models.Model(inputs=input_layer, outputs=x, name='nn_zero')

In [37]:
from tensorflow.contrib.keras import backend as K 

In [38]:
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1)) 

keras_nn.compile(optimizer=optimizers.Adam(lr=0.0001), loss=root_mean_squared_error, metrics=[root_mean_squared_error])

In [39]:
from tensorflow.contrib.keras import callbacks

In [40]:
batch_size = 16
epochs = 200

lr_scheduler = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=1, verbose=1, mode='auto', cooldown=0, min_lr=0.000001)
es = callbacks.EarlyStopping(monitor='val_loss', min_delta=0.00005, patience=3, verbose=0, mode='auto')

history = keras_nn.fit(X_train, y_train, 
          epochs=epochs,
          batch_size=batch_size,
          callbacks=[lr_scheduler, es],
          validation_data=(X_test, y_test))

Train on 3121 samples, validate on 1338 samples
Epoch 1/200
3121/3121 [==============================]3121/3121 [==============================] - 1s 448us/step - loss: 2.6035 - root_mean_squared_error: 2.6035 - val_loss: 1.5193 - val_root_mean_squared_error: 1.5193

Epoch 2/200
3121/3121 [==============================]3121/3121 [==============================] - 1s 303us/step - loss: 1.4883 - root_mean_squared_error: 1.4883 - val_loss: 1.4702 - val_root_mean_squared_error: 1.4702

Epoch 3/200
3121/3121 [==============================]3121/3121 [==============================] - 1s 305us/step - loss: 1.4301 - root_mean_squared_error: 1.4301 - val_loss: 1.4343 - val_root_mean_squared_error: 1.4343

Epoch 4/200
3121/3121 [==============================]3121/3121 [==============================] - 1s 305us/step - loss: 1.4018 - root_mean_squared_error: 1.4018 - val_loss: 1.4564 - val_root_mean_squared_error: 1.4564

Epoch 5/200
3056/3121 [============================>.]3056/3121 [=======

In [41]:
pred_keras = np.expm1(keras_nn.predict(X_test_scala))

In [42]:
sub = pd.read_csv('input/sample_submission.csv')
sub["target"] = pred_keras

print(sub.head())
sub.to_csv('submission.csv', index=False)

          ID     target
0  000137c73  1867677.0
1  00021489f  1867677.0
2  0004d7953  2100723.0
3  00056a333  7193542.0
4  00056d8eb  1867677.0
